Установка дополнительных библиотек

In [1]:
%system
! y|conda install psycopg2

/bin/sh: 1: y: not found
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.8
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - psycopg2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    krb5-1.16.1                |       hbb41f41_0         1.4 MB  conda-forge
    ca-certificates-2018.8.13  |       ha4d7672_0         135 KB  conda-forge
    certifi-2018.8.13          |           py36_0         138 KB  conda-forge
    libpq-9.6.3                |                0          97 KB  conda-forge
    psycopg2-2.7.5             |   py36hdffb7b8_1         509 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following 

## Подключение к базе данных Postgres

In [14]:
import psycopg2
import os

params = {
    "host": os.environ['APP_POSTGRES_HOST'],
    "port": os.environ['APP_POSTGRES_PORT'],
    "user": 'postgres'
}
conn = psycopg2.connect(**params)
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [15]:
import pandas as pd

In [17]:
cars_sql = "SELECT * FROM d_car"
accident_type_sql = "SELECT * FROM d_accident_type"
accident_sql = "SELECT * FROM f_accident"
participant_sql = "SELECT * FROM d_participant"

#select from cars
cursor.execute(cars_sql)
cars = pd.DataFrame([a for a in cursor.fetchall()],
                        columns=['id', 'brand', 'car_model',
                                    'color', 'manufacture_year'])
#select from accident_type
cursor.execute(accident_type_sql)
accident_types = pd.DataFrame([a for a in cursor.fetchall()],
                        columns=['id', 'name', 'alias'])
#select from accidents
cursor.execute(accident_sql)
accidents = pd.DataFrame([a for a in cursor.fetchall()],
                        columns=['id', 'datetime', 'address',
                                'longitude','latitude','dead',
                                'injured','type_id','alias',
                                'scheme','participants','geo_updated',
                                'created_at'])
#select from participant
cursor.execute(participant_sql)
participants = pd.DataFrame([a for a in cursor.fetchall()],
                        columns=['id','role','driving_experience',
                                    'status','gender','abscond',
                                    'car_id','mvc_id'])
conn.commit()

## Использование Pandas для работы с данными

1. Выбрать TOP-5 марок машины, с которыми чаще всего происходит ДТП
2. Для каждого участника ДТП выбрать количество общее количество участников, количество машин, учавствовавших в ДТП и тип участника
3. Установить, с какой средней частотой происходят аварии на различных улицах москвы.
4. Выбрать все ДТП, в которых участвовали животные
5. Выбрать все ДТП в которых погиб пассажир 

In [37]:
# 1. Выбрать TOP-5 марок машины, с которыми чаще всего происходит ДТП
# SELECT d_car.brand, COUNT(d_car.id)
#  FROM public.d_participant 
#        LEFT JOIN public.d_car
#           ON d_participant.car_id = d_car.id
#           WHERE d_participant.role = 'Водитель'
#           GROUP BY d_car.brand
#           ORDER BY 2 DESC
#           LIMIT 5;

df = pd.merge(participants, cars, how='outer', left_on=['car_id'], right_on=['id'])
df[df['role']=='Водитель'].groupby('brand')[['id_y']].count().sort_values(by=['id_y'], ascending=False).head(5)

,id_y
brand,
ВАЗ,319
HYUNDAI,276
VOLKSWAGEN,208
KIA,203
MERCEDES,197


In [55]:
# 2. Для каждого участника ДТП выбрать количество общее количество участников, количество машин, 
# учавствовавших в ДТП и тип участника
# SELECT d_participant.mvc_id,
#        d_participant.role,
#        d_participant.gender,
#        d_participant.status,
#        COUNT(1) OVER (PARTITION BY mvc_id) as count_participants,
#        SUM(CASE WHEN d_participant.role = 'Водитель' then 0 else 1 end) 
#            OVER (PARTITION BY mvc_id, car_id) as count_passengers,
#        SUM(CASE WHEN d_participant.role = 'Водитель' then 1 else 0 end)
#            OVER (PARTITION BY mvc_id) as count_cars
#FROM public.d_participant;
df = participants\
            .assign(is_driver = [1 if x != 'Водитель' else 0 for x in df['role']])\
            .assign(is_passenger = [0 if x != 'Водитель' else 1 for x in df['role']])\

df.assign(count_participants = df.groupby('mvc_id')['id'].transform('count'))\
  .assign(count_passengers = df.groupby('mvc_id')['is_passenger'].transform('sum'))\
  .assign(count_cars = df.groupby('mvc_id')['is_driver'].transform('sum'))\
  .sort_values('mvc_id')[['mvc_id', 'role', 'gender', 'status',
                          'count_participants','count_passengers', 'count_cars' ]]\
  .head(20)


,mvc_id,role,gender,status,count_participants,count_passengers,count_cars
0,1,Пешеход,женщина,"Раненый, находящийся (находившийся) на стацион...",1,0,1
1,2,Водитель,мужчина,Не пострадал,2,2,0
2,2,Водитель,мужчина,"Раненый, находящийся (находившийся) на амбула...",2,2,0
3,3,Водитель,None,,2,1,1
4,3,Пешеход,женщина,"Раненый, находящийся (находившийся) на стацион...",2,1,1
5,4,Водитель,мужчина,Скончался в течение 1 суток,1,1,0
6,5,Водитель,женщина,Не пострадал,2,1,1
7,5,Пешеход,женщина,"Раненый, находящийся (находившийся) на амбула...",2,1,1
8,6,Водитель,мужчина,"Раненый, находящийся (находившийся) на амбула...",1,1,0
9,7,Пассажир,мужчина,"Раненый, находящийся (находившийся) на стацион...",3,2,1


In [141]:
# 3. Установить, с какой средней частотой происходят аварии на различных улицах Mосквы
"""
WITH t as (
SELECT split_part(address, ',', 2) as street,
            COALESCE(LEAD(datetime) OVER 
                    (PARTITION BY split_part(address, ',', 1), 
                                   split_part(address, ',', 2)
                     ORDER BY datetime
                    ), datetime)
            - datetime as interval_between
FROM public.f_accident
        WHERE address IS NOT NULL
              AND lower(split_part(address, ',', 1)) like '%москва%')
SELECT street, AVG(interval_between) as avg_interval FROM t
        WHERE interval_between > 0 * interval '1 day'
            GROUP BY street
            ORDER BY 2;
"""
df = accidents.assign(street = accidents['address'].str.split(',', expand=True)[1])\
              .assign(region = accidents['address'].str.split(',', expand=True)[0])\
              .sort_values('datetime')
df.groupby(['street']).datetime.apply(lambda x: x.diff().mean()).sort_values().head(5)


street
 А-101                                                                                      3 days 06:16:00
 А-106 Рублево-Успенское шоссе                                                       5 days 07:53:45.941422
 ул Летчика Грицевца (п Внуковское)                                                         9 days 13:03:00
 ул Александры Монаховой                                                            14 days 04:12:13.333333
 М-3 Украина Москва - Калуга - Брянск - граница с Украиной (основное направление)   15 days 13:16:47.142857
Name: datetime, dtype: timedelta64[ns]

In [156]:
# 4. Выбрать все ДТП, в которых участвовали животные
"""
SELECT f_accident.id, f_accident.datetime, f_accident.longitude, 
            f_accident.latitude, d_accident_type.name
FROM public.f_accident
        LEFT JOIN public.d_accident_type
            ON f_accident.type_id = d_accident_type.id
WHERE name = 'Наезд на животное';
"""
df=accidents.merge(accident_types, how='left', left_on=['type_id'], right_on=['id'])
df[df['name']=='Наезд на животное'][['id_x', 'datetime', 'longitude', 'latitude', 'name']]

,id_x,datetime,longitude,latitude,name
1873,1529,2016-10-17 17:15:00+00:00,37.3081,55.5575,Наезд на животное


In [175]:
# 5. Выбрать все ДТП в которых погиб пассажир
"""
WITH t as
(SELECT mvc_id, 
             SUM(case when lower(status) like '%скончался%' then 1 else 0 end) as count_death
        FROM public.d_participant
        GROUP BY mvc_id)
SELECT f_accident.id, datetime, longitude, latitude, count_death 
FROM public.f_accident
        LEFT JOIN t 
            ON f_accident.id = t.mvc_id
            WHERE count_death > 0;
"""
df = participants.assign(count_death = [0 if x.lower().find("скончался") == -1 else 1 for x in participants['status']])\
        .groupby('mvc_id').sum()[['id', 'count_death']]
outdata = accidents.merge(df[df['count_death']>0], how='inner', left_on=['id'], right_on=['mvc_id'])\
        [['id_x','datetime', 'longitude', 'latitude', 'count_death']].sort_values('id_x')
outdata.head(5)

,id_x,datetime,longitude,latitude,count_death
2,4,2016-11-19 16:55:00+00:00,37.114981,55.716298,1
3,11,2017-01-07 07:10:00+00:00,37.309206,55.747800,1
22,18,2015-09-10 14:55:00+00:00,37.355833,55.793889,1
9,20,2015-09-05 17:15:00+00:00,37.300000,55.797500,1
24,35,2015-06-25 10:50:00+00:00,37.306389,55.799167,1


## Выгрузка результатов аналитического запроса (Pandas)

In [176]:
outdata.to_csv('../data/output/data.csv')